In [2]:
!pip install -q tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python
!pip install -q matplotlib

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-metadata 1.5.0 requires absl-py<0.13,>=0.9, but you'll have absl-py 0.15.0 which is incompatible.
grpcio-status 1.43.0 requires grpcio>=1.43.0, but you'll have grpcio 1.41.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [6]:
spec = model_spec.get('efficientdet_lite0') # maybe different lite1-4
spec.model_dir="/logs/fit/test/"

In [7]:
def read_train_val_test(dataset_dir, label_map):
    return tuple(map(lambda set_: object_detector.DataLoader.from_pascal_voc(images_dir=os.path.join(dataset_dir, set_), annotations_dir=os.path.join(dataset_dir, set_), label_map=label_map), ("train", "val", "test")))

In [8]:
text_file = open("./assets/list_promis.txt", "r")
query_array = text_file.read().split(',\n')
print(query_array)

['LewisHamilton', 'SebastianVettel', 'MaxVerstappen', 'KarlLauterbach', 'CristianoRonaldo', 'LionelMessi', 'EdSheeran', 'TomCruise', 'HeleneFischer', 'ArnoldSchwarzenegger', 'GüntherJauch', 'StefanRaab', 'HeidiKlum', 'ManuelNeuer', "ElyasM'barek", 'JudithRakers', 'AngelaMerkel', 'WladimirPutin', 'OlafScholz', 'JoeBiden', 'DonaldTrump', 'ChristianLindner', 'AnnalenaBaerbock', 'ElonMusk', 'JeffBezos', 'BarackObama', 'ChrisPratt', 'XiJinping', 'DwayneJohnson', 'MacKenzieScott', 'AnthonyFauci', 'GeorgeClooney', 'BradPitt', 'KobeBryant', "ShaquilleO'Neal", 'VinDiesel', 'JasonMomoa', 'ChrisHemsworth', 'TomHiddleston', 'BenedictCumberbatch', 'ChadwickBoseman', 'LeBronJames', 'MichaelJordan', 'MichaelJackson', 'LeonardoDiCaprio', 'TomHanks', 'WillSmith', 'DenzelWashington', 'RobertDowneyJr', 'TomHardy', 'MorganFreeman', 'NovakDjokovic', 'RogerFederer', 'ChristianBale', 'SylvesterStallone', 'AnnaFaris', 'AngelinaJolie', 'HillaryClinton', 'JustinBieber', 'RyanGosling', 'EllenDegeneres', 'LadyGag

In [9]:
from datetime import datetime

path = os.getcwd()
abspath = os.path.abspath(os.path.join(path, os.pardir))

datapath = abspath + "/datasetV2/"
outpath =  f'{abspath}/models/TF_V3/'

if not os.path.exists(outpath):
    print(outpath)
    os.mkdir(outpath)

In [10]:
labelmap = query_array
train_data, validation_data, test_data = read_train_val_test(datapath, labelmap)

KeyError: 'RafaelNadal'

In [7]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
%tensorboard --logdir /logs/fit/test/ --host 0.0.0.0

In [9]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, validation_data=validation_data, epochs=5)

Epoch 1/5
738/738 [==============================] - 367s 477ms/step - det_loss: 1.3986 - cls_loss: 1.2096 - box_loss: 0.0038 - reg_l2_loss: 0.0729 - loss: 1.4714 - learning_rate: 0.0140 - gradient_norm: 1.4132 - val_det_loss: 0.9516 - val_cls_loss: 0.8561 - val_box_loss: 0.0019 - val_reg_l2_loss: 0.0730 - val_loss: 1.0247
Epoch 2/5
637/738 [========================>.....] - ETA: 45s - det_loss: 0.9154 - cls_loss: 0.8240 - box_loss: 0.0018 - reg_l2_loss: 0.0730 - loss: 0.9884 - learning_rate: 0.0142 - gradient_norm: 1.3333

KeyboardInterrupt: 

In [10]:
model.evaluate(test_data)

39/39 [==============================] - 212s 4s/step



{'AP': 0.66493195,
 'AP50': 0.77795243,
 'AP75': 0.75517076,
 'APs': 0.2176599,
 'APm': 0.48967555,
 'APl': 0.69998926,
 'ARmax1': 0.74306005,
 'ARmax10': 0.8323864,
 'ARmax100': 0.83526784,
 'ARs': 0.38333333,
 'ARm': 0.7039489,
 'ARl': 0.85815537,
 'AP_/LewisHamilton': 0.54521936,
 'AP_/SebastianVettel': 0.5841006,
 'AP_/MaxVerstappen': 0.69106203,
 'AP_/KarlLauterbach': 0.8064475,
 'AP_/CristianoRonaldo': 0.7100038,
 'AP_/LionelMessi': 0.5224295,
 'AP_/EdSheeran': 0.7760605,
 'AP_/TomCruise': 0.6955497,
 'AP_/HeleneFischer': 0.6301009,
 'AP_/ArnoldSchwarzenegger': 0.6310288,
 'AP_/GüntherJauch': 0.700951,
 'AP_/StefanRaab': 0.5042378,
 'AP_/HeidiKlum': 0.69642603,
 'AP_/ManuelNeuer': 0.7010178,
 "AP_/ElyasM'barek": 0.8099643,
 'AP_/JudithRakers': 0.5114743,
 'AP_/AngelaMerkel': 0.76910394,
 'AP_/WladimirPutin': 0.65886945,
 'AP_/OlafScholz': 0.76780844,
 'AP_/JoeBiden': 0.71000177,
 'AP_/DonaldTrump': 0.7024872,
 'AP_/ChristianLindner': 0.6318615,
 'AP_/AnnalenaBaerbock': 0.88129455

In [11]:
model.export(export_dir=outpath)

In [ ]:
model.evaluate_tflite(outpath + 'model.tflite', validation_data)

In [3]:
import time

import cv2
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def to_json(self):
        return {
            "x": self.x,
            "y": self.y,
        }

    def __str__(self):
        return str(self.to_json())

class BoundingBox:
    def __init__(self, xmin, ymin, xmax, ymax):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.center = Point((xmin + xmax) // 2, (ymin + ymax) // 2)
        self.width = xmax - xmin
        self.height = ymax - ymin
        self.area = self.width * self.height

    def to_json(self):
        return {
            "xmin": self.xmin,
            "ymin": self.ymin,
            "xmax": self.xmax,
            "ymax": self.ymax,
        }

    def __str__(self):
        return str(self.to_json())

class Detection:
    def __init__(self, class_, confidence, xmin, ymin, xmax, ymax):
        self.class_ = class_
        self.confidence = confidence
        self.bounding_box = BoundingBox(xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax)

    def to_json(self):
        return {
            "class": self.class_,
            "confidence": self.confidence,
            "box": self.bounding_box.to_json(),
        }

    def __str__(self):
        return str(self.to_json())


class EfficientDetLite:
    def __init__(self, model_path, labelmap):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()

        # iti = input tensor index
        input_details = self.interpreter.get_input_details()
        self.iti_image = input_details[0]["index"]
        self.it_image_dtype = input_details[0]["dtype"]
        self.it_image_width = input_details[0]["shape"][2]
        self.it_image_height = input_details[0]["shape"][1]

        # oti = output tensor index
        output_details = self.interpreter.get_output_details()
        self.oti_bounding_boxes = output_details[1]["index"]
        self.oti_classes = output_details[3]["index"]
        self.oti_confidences = output_details[0]["index"]

        self.labelmap = labelmap

    def detect(self, image, threshold=0):
        input_tensor = self.prepare_image(image)
        self.interpreter.set_tensor(self.iti_image, input_tensor)
        self.interpreter.invoke()

        bounding_boxes = self.interpreter.get_tensor(self.oti_bounding_boxes)[0]
        classes = self.interpreter.get_tensor(self.oti_classes)[0]
        confidences = self.interpreter.get_tensor(self.oti_confidences)[0]
        detections = zip(classes, bounding_boxes, confidences)
        detections = filter(lambda x: x[2] >= threshold, detections)

        def tensors2detection(x):
            def clamp(n, min_, max_):
                return max(min(max_, n), min_)

            xmin = clamp(int(round(x[1][1] * image.shape[1])), 0, image.shape[1])
            ymin = clamp(int(round(x[1][0] * image.shape[0])), 0, image.shape[0])
            xmax = clamp(int(round(x[1][3] * image.shape[1])), 0, image.shape[1])
            ymax = clamp(int(round(x[1][2] * image.shape[0])), 0, image.shape[0])
            name = str(self.labelmap[int(x[0])])
            confidence = float(x[2])

            detection = Detection(name, confidence, xmin, ymin, xmax, ymax)
            return detection

        detections = map(lambda x: tensors2detection(x), detections)
        detections = list(detections)
        return detections

    def prepare_image(self, image):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (self.it_image_width, self.it_image_height), interpolation=cv2.INTER_NEAREST)
        tensor = np.reshape(image, (1, self.it_image_height, self.it_image_width, 3))
        tensor = tensor.astype(self.it_image_dtype)
        return tensor

def show_image(image, label=None, scale=1):
    show_images([image], labels=[label] if label is not None else None, scale=scale)


def show_images(images, labels=None, scale=1):
    figsize = plt.rcParams["figure.figsize"]
    plt.figure(figsize=(figsize[0] * scale, figsize[1] * scale))
    for i in range(len(images)):
        plt.subplot(1, len(images), i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        image = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
        plt.imshow(image, cmap=plt.cm.binary)
        if labels is not None:
            plt.xlabel(labels[i])
    plt.show()


def show_detection(image, detection, scale=1, thickness=1, colors=None, confidence=True):
    show_image(draw_detections(image,
                               [detection],
                               scale=scale,
                               thickness=thickness,
                               colors=colors,
                               confidence=confidence))


def show_detections(image, detections, scale=1, thickness=1, colors=None, confidence=True):
    show_image(draw_detections(image,
                               detections,
                               scale=scale,
                               thickness=thickness,
                               colors=colors,
                               confidence=confidence))


def draw_detection(image, detection, scale=1, thickness=1, colors=None, confidence=True):
    return draw_detections(image, [detection], scale, thickness, colors, confidence)


def draw_detections(image, detections, scale=1, thickness=1, colors=None, confidence=True):
    colors = {} if colors is None else {k: tuple(reversed(v)) for k, v in colors.items()}
    result = image.copy()
    for detection in detections:
        print(detection)
        xmin = int(detection.bounding_box.xmin)
        ymin = int(detection.bounding_box.ymin)
        xmax = int(detection.bounding_box.xmax)
        ymax = int(detection.bounding_box.ymax)
        color = colors.get(detection.class_)
        if color is None:
            color = (0, 0, 0)
        result = cv2.rectangle(result, (xmin, ymin), (xmax, ymax), color, thickness)
        if confidence:
            text = "{:.2f}".format(float(detection.confidence))
            #text = f"{detection.class}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            textsize = cv2.getTextSize(text, font, scale, thickness)[0]
            result = cv2.putText(result,
                                 text,
                                 (
                                     ((xmin + xmax) - textsize[0]) // 2,
                                     ((ymin + ymax) + textsize[1]) // 2
                                 ),
                                 font,
                                 scale,
                                 color,
                                 thickness,
                                 cv2.LINE_AA)
    return result

In [12]:
model = EfficientDetLite(outpath + 'model.tflite', labelmap)

In [4]:
image = cv2.imread(datapath + 'test/001_4d0d9a57.jpg') # change to exisiting image

print(image)

detections = model.detect(image, threshold=0.6)

for detection in detections:
  print(detection)

show_detections(image, detections, thickness=4)

NameError: name 'datapath' is not defined